### Exercicio de WithColumn 2


1. Criar um dataframe para ler o arquivo no HDFS /user/<nome/data/juros_selic/juros_selic

In [25]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [17]:
!hdfs dfs -ls /user/clayton/data/juros_selic/juros_selic

-rw-r--r--   3 root supergroup       7954 2022-04-05 16:11 /user/clayton/data/juros_selic/juros_selic


1. Criar um dataframe para ler o arquivo no HDFS /user/<nome/data/juros_selic/juros_selic

In [20]:
df_juros_selic = spark.read.csv("/user/clayton/data/juros_selic/juros_selic", header="true", sep=";")
df_juros_selic.show(2)

+----------+-----+
|      data|valor|
+----------+-----+
|01/06/1986| 1,27|
|01/07/1986| 1,95|
+----------+-----+
only showing top 2 rows



In [29]:
df_juros_ano = df_juros_selic.withColumn("ano", split(col("data"),"/").getItem(2))
#convertendo para float p poder fazer a soma
df_juros_valor = df_juros_ano.withColumn("valor", regexp_replace(col("valor"),"\,","\.").cast(FloatType()))


In [30]:
df_juros_valor.printSchema()

root
 |-- data: string (nullable = true)
 |-- valor: float (nullable = true)
 |-- ano: string (nullable = true)



2. Agrupar todas as datas pelo ano em ordem decrescente e salvar a quantidade de meses ocorridos, o valor médio, mínimo e máximo do campo valor com a seguinte estrutura:

In [42]:
juros_relatorio = df_juros_valor.groupBy("ano").agg(count("ano").alias("Meses"),format_number(avg("valor"),2).alias("Valor médio"), min("valor").alias("Valor Minimo"), max("valor").alias("Valor Máximo")).sort(desc("ano"))

3. Salvar no hdfs:///user/<nome>/relatorioAnual com compressão zlib e formato orc


In [98]:
juros_relatorio.write.orc("/user/clayton/relatorioAnualParticionado",partitionBy="ano",compression="zlib")

In [116]:
!hdfs dfs -ls /user/clayton/relatorioAnualParticionado | grep 2018

drwxr-xr-x   - root supergroup          0 2022-04-07 19:28 /user/clayton/relatorioAnualParticionado/ano=2018


In [101]:
juros_renamed = juros_relatorio.withColumnRenamed("Valor médio", "valor_medio").withColumnRenamed("Valor Máximo","valor_maximo").withColumnRenamed("Valor Minimo","valor_minimo")


In [103]:
juros_renamed.write.saveAsTable("relatorioAnualParticionado",partitionBy="ano", mode="Overwrite")

In [104]:
spark.read.table("relatorioAnualParticionado")

DataFrame[Meses: bigint, valor_medio: string, valor_minimo: float, valor_maximo: float, ano: string]

In [105]:
spark.sql("select * from relatorioAnualParticionado where ano = 2018").show()

+-----+-----------+------------+------------+----+
|Meses|valor_medio|valor_minimo|valor_maximo| ano|
+-----+-----------+------------+------------+----+
|   12|       0.52|        0.47|        0.58|2018|
+-----+-----------+------------+------------+----+



In [109]:
!hdfs dfs -ls /user/hive/warehouse/relatorioanualparticionado

Found 35 items
-rw-r--r--   2 root supergroup          0 2022-04-07 19:30 /user/hive/warehouse/relatorioanualparticionado/_SUCCESS
drwxr-xr-x   - root supergroup          0 2022-04-07 19:30 /user/hive/warehouse/relatorioanualparticionado/ano=1986
drwxr-xr-x   - root supergroup          0 2022-04-07 19:30 /user/hive/warehouse/relatorioanualparticionado/ano=1987
drwxr-xr-x   - root supergroup          0 2022-04-07 19:30 /user/hive/warehouse/relatorioanualparticionado/ano=1988
drwxr-xr-x   - root supergroup          0 2022-04-07 19:30 /user/hive/warehouse/relatorioanualparticionado/ano=1989
drwxr-xr-x   - root supergroup          0 2022-04-07 19:30 /user/hive/warehouse/relatorioanualparticionado/ano=1990
drwxr-xr-x   - root supergroup          0 2022-04-07 19:30 /user/hive/warehouse/relatorioanualparticionado/ano=1991
drwxr-xr-x   - root supergroup          0 2022-04-07 19:30 /user/hive/warehouse/relatorioanualparticionado/ano=1992
drwxr-xr-x   - root supergroup          0 2022-04-07 19:3